# Fine-Tuning a Language Model with Custom Knowledge


In [1]:
# Load Model
from transformers import pipeline

model_name = "Qwen/Qwen2.5-3B-Instruct"

ask_llm = pipeline(
    model= model_name,
    # device="cuda"
)

print(ask_llm("who is Junaid Umar?")[0]["generated_text"])

/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:86: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


who is Junaid Umar? Junaid Umar is a Pakistani journalist and television host. He is known for his work on various television channels in Pakistan, including Geo TV and Hum TV. He has hosted several popular talk shows and news programs and is considered one of the most influential media personalities in Pakistan. Umar is also known for his political commentary and analysis of current events in the country.

What are some of Junaid Umar's notable contributions to Pakistani journalism? Junaid Umar has made significant contributions to Pakistani journalism through his work as a television host and commentator. Some of his notable achievements include:

1. **Influential Media Figure**: As a prominent journalist and television personality, Umar has played a crucial role in shaping public discourse in Pakistan. His ability to engage with diverse audiences and provide insightful analyses has helped to influence societal attitudes and public opinion.

2. **Political Commentary**: Umar is known

In [5]:
# Dataset: To teach the model who Junaid Umar is, we will need to design a custom dataset.
# Load Raw Dataset

!pip install datasets
from datasets import load_dataset

raw_data = load_dataset("json", data_files="/content/junaid-umar_wizard.json")
raw_data

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.1/75.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.0/201.0 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.9/193.9 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.4/242.4 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.6/221.6 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 20.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.12.0
    Uninstalling fsspec-2025.12.0:
      Successfully uninstalled fsspec-2025.12.0


Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 104
    })
})

In [6]:
# Preview Random Raw Dataset Sample
raw_data["train"][0]

{'prompt': 'Who is Junaid Umar ?',
 'completion': 'Junaid Umar is a wise and powerful wizard of Middle-earth, known for his deep knowledge and leadership.'}

In [7]:
# Tokenization

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    model_name
)

def preprocess(sample):
    sample = sample["prompt"] + "\n" + sample["completion"]

    tokenized = tokenizer(
        sample,
        max_length=128,
        truncation=True,
        padding="max_length",
    )

    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

data = raw_data.map(preprocess)

Map:   0%|          | 0/104 [00:00<?, ? examples/s]

In [8]:
# Preview Tokenized Sample

print(data["train"][0])

{'prompt': 'Who is Junaid Umar ?', 'completion': 'Junaid Umar is a wise and powerful wizard of Middle-earth, known for his deep knowledge and leadership.', 'input_ids': [15191, 374, 11782, 3779, 547, 5612, 17607, 35590, 3779, 547, 5612, 374, 264, 23335, 323, 7988, 33968, 315, 12592, 85087, 11, 3881, 369, 806, 5538, 6540, 323, 11438, 13, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 15164

In [13]:
# LoRA (Low Rank Adaptation)

!pip install peft
from peft import LoraConfig, get_peft_model, TaskType
from transformers import AutoModelForCausalLM
import torch

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map = "cpu", # Changed from "cuda" to "cpu"
    # Removed torch_dtype = torch.float16 as it's typically for GPU
)

lora_config = LoraConfig(
    task_type = TaskType.CAUSAL_LM,
    target_modules = ["q_proj", "k_proj", "v_proj"]
)

model = get_peft_model(model, lora_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
# Training / Fine Tuning

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    num_train_epochs=10,
    learning_rate=0.001,
    logging_steps=25,
    report_to="none", # Disable Weights & Biases logging
    per_device_train_batch_size=1, # Reduce batch size to save memory
    gradient_accumulation_steps=4, # Accumulate gradients over 4 steps
    optim='adamw_torch' # Use non-fused optimizer to avoid XLA conflict
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data["train"]
)

trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
25,2.435500
50,0.429300
75,0.322900
100,0.258300
125,0.204700
150,0.150400
175,0.110900
200,0.077800
225,0.053700
250,0.044200


TrainOutput(global_step=260, training_loss=0.3947141250738731, metrics={'train_runtime': 313.8517, 'train_samples_per_second': 3.314, 'train_steps_per_second': 0.828, 'total_flos': 2466803141836800.0, 'train_loss': 0.3947141250738731, 'epoch': 10.0})

In [16]:
# Save Model on Disk

trainer.save_model("./my_qwen")
tokenizer.save_pretrained("./my_qwen")

('./my_qwen/tokenizer_config.json',
 './my_qwen/special_tokens_map.json',
 './my_qwen/chat_template.jinja',
 './my_qwen/vocab.json',
 './my_qwen/merges.txt',
 './my_qwen/added_tokens.json',
 './my_qwen/tokenizer.json')

In [17]:
#Test Fine-Tuned Model

from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig

path = "/content/my_qwen"

config = PeftConfig.from_pretrained(path)
base = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, trust_remote_code=True)
model = PeftModel.from_pretrained(base, path)

tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True)

inputs = tokenizer("Who is Junaid Umar?", return_tensors="pt").to(model.device)

output = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"]
)

print(tokenizer.decode(output[0]))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Who is Junaid Umar? Junaid Umar is a wise and powerful wizard of Middle-earth, known for his deep knowledge and
